# 02 Build CLI Train/Predict

Extend the CLI to control features/models and generate artifacts.


## Table of Contents
- [Training CLI](#training-cli)
- [Prediction CLI](#prediction-cli)
- [Artifacts](#artifacts)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Model ops notebooks turn your work into reproducible runs with saved artifacts.
The goal is: someone else can run your pipeline and see the same metrics.


## Prerequisites (Quick Self-Check)
- Completed earlier modeling notebooks (regression/classification).
- Comfort running scripts and inspecting files under `outputs/`.

## What You Will Produce
- outputs/<run_id>/model.joblib
- outputs/<run_id>/metrics.json
- outputs/<run_id>/predictions.csv

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.
- You can point to the concrete deliverable(s) listed below and explain how they were produced.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Not recording which dataset/config a model was trained on.
- Overwriting artifacts without run IDs.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/05_model_ops/02_build_cli_train_predict.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/05_model_ops/02_build_cli_train_predict.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Practice model ops by extending the CLI scripts:
- add flags/config controls
- generate artifact bundles
- run predict to produce new outputs

This notebook is hands-on *engineering*, not just analysis.



## Primer: CLI design with `argparse` (making scripts configurable)

Scripts in this project use Python's `argparse` module to accept command-line arguments. This makes experiments controllable without editing code.

### Minimal argparse pattern

```python
import argparse

parser = argparse.ArgumentParser(description="Train recession classifier")
parser.add_argument("--config", type=str, required=True, help="Path to YAML config")
parser.add_argument("--model", type=str, default="logistic", choices=["logistic", "random_forest"])
parser.add_argument("--test-size", type=float, default=0.2)
args = parser.parse_args()

print(args.config, args.model, args.test_size)
```

Usage from the terminal:
```bash
python scripts/train_recession.py --config configs/recession.yaml --model random_forest --test-size 0.25
```

### Loading YAML configs

```python
import yaml

cfg = yaml.safe_load(open(args.config))
series = cfg["series"]       # which FRED series to use
features = cfg["features"]   # lag/rolling settings
```

The config + CLI flags together define a complete experiment specification.

### Saving run metadata

Every training run should save enough metadata to reproduce itself:

```python
import json
from datetime import datetime

run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
metadata = {
    "run_id": run_id,
    "config_path": str(args.config),
    "model_type": args.model,
    "test_size": args.test_size,
    "timestamp": run_id,
}
json.dump(metadata, open(f"outputs/{run_id}/run_metadata.json", "w"), indent=2)
```

### Common pitfalls

- Hard-coding values that should be flags (e.g., feature lists, split sizes).
- Not saving the config used for a run — making it impossible to reproduce.
- Using `sys.argv` directly instead of `argparse` (fragile, no help text, no validation).


<a id="training-cli"></a>
## Training CLI

### Goal
Extend `scripts/train_recession.py` so you can control key behavior from the command line.



### Your Turn (1): Inspect the current CLI


In [ ]:
# TODO: Open scripts/train_recession.py and find:
# - how argparse is set up
# - what args exist today
# - what config fields are read
...



### Your Turn (2): Add a meaningful flag


Implement at least one of these:
- `--model logistic|random_forest`
- `--include-gdp-features true|false`
- `--test-size 0.2`

Constraints:
- default behavior should remain unchanged
- the selected option must be written to the run folder (as JSON/YAML)



In [ ]:
# TODO: After implementing, re-run training and confirm it works.
# In terminal: python scripts/train_recession.py --config configs/recession.yaml --model random_forest
...



<a id="prediction-cli"></a>
## Prediction CLI

### Goal
Extend `scripts/predict_recession.py` to support useful output controls.



### Your Turn: Add a filter option


Implement one option:
- `--last-n 20` (only write last N predictions)
- `--from-date 2010-01-01` (filter by date)

Make sure the filter is applied to the output CSV.



In [ ]:
# TODO: After implementing, run prediction on your latest run.
# In terminal: python scripts/predict_recession.py --run-id <run_id> --last-n 20
...



<a id="artifacts"></a>
## Artifacts

### Goal
Verify your artifact bundle is complete and interpretable.



### Your Turn: Inspect the newest run artifacts


In [ ]:
from pathlib import Path

out_dir = PROJECT_ROOT / 'outputs'
runs = sorted([p for p in out_dir.glob('*') if p.is_dir()])
if not runs:
    raise RuntimeError('No runs found. Run training first.')

run = runs[-1]
print('run:', run.name)
print('files:', [p.name for p in run.iterdir()])
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Run one script end-to-end and confirm an artifact bundle exists.
# Example:
# - list outputs/ and pick the newest run_id
# - assert model.joblib and metrics.json exist
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Training CLI</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_build_cli_train_predict — Training CLI
# Reference idea: add argparse flags in scripts/train_recession.py
# - --include-gdp-features true/false
# - --model logistic|rf
# Then branch logic when selecting feature_cols and choosing estimator.
```

</details>

<details><summary>Solution: Prediction CLI</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_build_cli_train_predict — Prediction CLI
# Reference idea: add --last-n option to scripts/predict_recession.py
# Slice the output dataframe before writing.
```

</details>

<details><summary>Solution: Artifacts</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_build_cli_train_predict — Artifacts
# Ensure each run writes:
# - model.joblib
# - metrics.json
# - predictions.csv
# - run_metadata.json
```

</details>

